In [147]:
!pip install --quiet imbalanced-learn
!pip install --quiet xgboost


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('dataset.csv', index_col=0)

In [3]:
df.describe(percentiles=[0.05, 0.1, 0.15, 0.25, 0.75, 0.8, 0.9, 0.95])

,time,amount,age,income,reward,difficulty,duration,days_since_membership,offered_channels_count
count,303572.000000,268141.000000,303572.000000,269899.000000,298765.000000,298765.000000,298765.000000,303572.000000,303572.000000
mean,365.011391,13.974394,60.926087,64255.691944,4.414808,7.778046,6.594263,1456.799942,3.335212
std,200.508790,31.849983,26.085974,21223.742912,3.418464,5.014472,2.148365,421.682461,0.751638
min,0.000000,0.050000,18.000000,30000.000000,0.000000,0.000000,3.000000,209.000000,0.000000
5%,0.000000,0.660000,24.000000,33000.000000,0.000000,0.000000,3.000000,562.000000,2.000000
10%,48.000000,1.230000,29.000000,37000.000000,0.000000,0.000000,4.000000,939.000000,3.000000
15%,120.000000,1.820000,35.000000,40000.000000,0.000000,0.000000,4.000000,1018.000000,3.000000
25%,186.000000,3.290000,43.000000,48000.000000,2.000000,5.000000,5.000000,1176.000000,3.000000
50%,408.000000,10.750000,57.000000,62000.000000,5.000000,10.000000,7.000000,1563.000000,3.000000
75%,528.000000,19.400000,72.000000,77000.000000,5.000000,10.000000,7.000000,1790.000000,4.000000


In [4]:
df = df[(df.age < 90) & (df.income > 25_000) & (df.income < 100_000)]

In [5]:
df.isna().sum()

person                        0
event                         0
time                          0
amount                    27260
gender                        0
age                           0
income                        0
reward                     4023
difficulty                 4023
duration                   4023
offer_type                 4023
email                      4023
social                     4023
mobile                     4023
web                        4023
days_since_membership         0
offered_channels_count        0
dtype: int64

In [6]:
df.head(2)

,person,event,time,amount,gender,age,income,reward,difficulty,duration,offer_type,email,social,mobile,web,days_since_membership,offered_channels_count
offer id,,,,,,,,,,,,,,,,,
5a8bc65990b245e5a138643cd4eb9837,0009655768c64bdeb2e877511632db8f,offer received,168,NaN,M,33,72000.0,0.0,0.0,3.0,informational,True,True,True,False,1571,3
5a8bc65990b245e5a138643cd4eb9837,0009655768c64bdeb2e877511632db8f,offer viewed,192,NaN,M,33,72000.0,0.0,0.0,3.0,informational,True,True,True,False,1571,3


In [7]:
person_uniques = df.person.unique()
person_nuniques = len(person_uniques)
person_map = dict(zip(person_uniques, range(1, person_nuniques+1)))

with open('person_dict.json', 'w') as p:
    json.dump(person_map, p)

In [8]:
df.person = df.person.map(person_map)

In [9]:
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit_transform(df.event)

array([1, 2, 3, ..., 2, 3, 3])

In [10]:
event_to_labels = dict(zip(le.classes_, range(len(le.classes_))))
labels_to_event = dict(zip(event_to_labels.values(), event_to_labels.keys()))

with open('event_to_labels.json', 'w') as p:
    json.dump(event_to_labels, p)
with open('labels_to_event.json', 'w') as p:
    json.dump(labels_to_event, p)

In [11]:
df[['email', 'social', 'mobile', 'web']] = df[['email', 'social', 'mobile', 'web']].fillna(False).astype(int)

/tmp/ipykernel_7490/926569504.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['email', 'social', 'mobile', 'web']] = df[['email', 'social', 'mobile', 'web']].fillna(False).astype(int)


In [12]:
target = 'event'
y = df.groupby('person')[target].shift(-1)
X = df.drop(columns=target)

X = X[~y.isna()]
y.dropna(inplace=True)

tmp = pd.concat([X, y], axis=1)

In [13]:
train, test = train_test_split(tmp, test_size=0.2, random_state=42, stratify=tmp.time)

In [14]:
train.to_csv('train.csv', index=True)
test.to_csv('test.csv', index=True)